<a href="https://colab.research.google.com/github/saaransh13/MIDAS-IIITD-task/blob/master/CV_Problem/CV_Problem_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import train_test_split
import pickle
import cv2
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## Reading .pkl files

with open('your_drive/train_image.pkl', 'rb') as f: ## To be provided by the user
    train_image = pickle.load(f)

with open('your_drive/train_label.pkl', 'rb') as f: ## To be provided by the user
    train_label = pickle.load(f)

with open('your_drive/test_image.pkl', 'rb') as f: ## To be provided by the user
    test_image = pickle.load(f)
    

In [0]:
## Reshaping the train/test-images to (1,28,28) shape

for i,x in enumerate(train_image):
    train_image[i] = np.asarray(train_image[i]).reshape(1,28,28)

for i,x in enumerate(test_image):
    test_image[i] = np.asarray(test_image[i]).reshape(1,28,28)

train_image = np.array(train_image)
test_image = np.array(test_image)


In [0]:
## Preprocessing the training label.

train_label = np.array(train_label)

for i,x in enumerate(train_label):
##    if x == 0:
##        train_label[i] = 0
    if x == 2:
        train_label[i] = 1
    elif x == 3:
        train_label[i] = 2
    elif x == 6:
        train_label[i] = 3
## 0 : 0 , 1 : 2, 2 : 3, 3 : 6 -- changing class value for OHE (One Hot Encoding)

train_label = np_utils.to_categorical(train_label, 4) # Array to OHE


In [0]:
## Splitting the data into train & validation data.

X_train, X_val, Y_train, Y_val = train_test_split(train_image, train_label, test_size=0.30)


In [0]:
## CNN model. 

model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(1,28,28)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
#model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation = 'softmax'))


In [0]:
## Compiling the model

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
## Training the model 

model.fit(X_train, Y_train, 
          batch_size=32, epochs=35, verbose=1)

Epoch 1/35
5600/5600 [==============================] - 18s 3ms/step - loss: 0.7212 - acc: 0.7059
Epoch 2/35
5600/5600 [==============================] - 17s 3ms/step - loss: 0.4728 - acc: 0.8077
Epoch 3/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.4106 - acc: 0.8291
Epoch 4/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.3493 - acc: 0.8523
Epoch 5/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.2997 - acc: 0.8768
Epoch 6/35
5600/5600 [==============================] - 17s 3ms/step - loss: 0.2641 - acc: 0.8938
Epoch 7/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.2319 - acc: 0.9111
Epoch 8/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.2102 - acc: 0.9141
Epoch 9/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.1728 - acc: 0.9320
Epoch 10/35
5600/5600 [==============================] - 16s 3ms/step - loss: 0.1440 - acc: 0.9452
Epoch 11/35
5600/56

In [46]:
## Evaluating the data on val-dataset

score = model.evaluate(X_val, Y_val, verbose=1)

print(score)


2400/2400 [==============================] - 3s 1ms/step
[0.7151526321967443, 0.8725]


In [47]:
## Making prediction on the training-dataset

Y_test = model.predict(test_image, verbose=1)


2000/2000 [==============================] - 3s 1ms/step


In [0]:
## Converting OHE to Array

final_y_pred = []
for i,x in enumerate(Y_test):
    final_y_pred.append(np.argmax(Y_test[i]))


In [0]:

## Setting back the original values of the classes

for i,x in enumerate(final_y_pred):
    if x == 1:
        final_y_pred[i] = 2
    elif x == 2:
        final_y_pred[i] = 3
    elif x == 3:
        final_y_pred[i] = 6
## 0 : 0 , 1 : 2, 2 : 3, 3 : 6  


In [0]:
## Saving the final solution.csv file
raw_data = {'class':final_y_pred}
df = pd.DataFrame(raw_data, columns = ['class'])
df.index.name = 'image_index'
df.to_csv('your_drive/Saaransh_Pandey.csv')  ## To be provided by the user 